In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from azure.storage.blob import BlobService
import pyspark
#from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import time
from math import sqrt
from operator import add
conf = (SparkConf().set("spark.driver.maxResultSize", "8g"))
print conf.get("spark.num-executors")
conf = (SparkConf().set("spark.num-executors", "4"))
conf = (SparkConf().set("spark.driver-cores", "4"))
conf = (SparkConf().set("spark.num-executors", "40"))
conf = (SparkConf().set("spark.executor-cores", "4"))
conf = (SparkConf().set("spark.executor-memory", "16G"))
conf = (SparkConf().set("spark.driver-memory", "16G"))
spark = SparkSession.builder.master("local").appName("do_mf").config(conf=conf).getOrCreate()
print conf.get("spark.driver-memory")
sc.setLogLevel('ERROR')

Read data

In [ ]:
block_blob_service_sample = BlobService(account_name='asoscustprofdevelopment', account_key='X1nmVm/GGg+UQLrDHBTfqxNRFvD4XI1GjnMCF2C2DvWT8pu+3qVI3BVt+2t2k0Ksz1PNPRoHgpqC9QOxmmZJcg==')

In [ ]:
viewSchema = StructType([StructField("idd", IntegerType(), False), StructField("cid", IntegerType(), False),
                             StructField("pid", IntegerType(), False),
                             StructField("rating", IntegerType(), False)])
                             
viewings = sc.textFile("wasbs://sdcsparkrd01@asoscustprofdevelopment.blob.core.windows.net/test/customer_product_rating.csv").map(lambda x: x.split(',')).map(lambda l: Rating(int(l[1]), int(l[2]), float(l[3])))


Do MF

In [ ]:
ranks = [2, 4, 8, 16, 32, 64, 128, 256, 512]
for i in range(0, len(ranks)):
    rank = ranks[i]
    numIterations = 5
    reg = 0.001
    alpha = 50
    print("Train the model...")
    t_start = time.time()
    model = ALS.trainImplicit(viewings, rank, numIterations, reg, alpha)
    t_stop= time.time()
    print t_stop - t_start
    model = model.userFeatures()
    model = model.map(lambda (x, y) : (x, ' '.join(str(e) for e in y)))
    fname = "mf_" + rank[i]
    model.toDF().coalesce(1).write.csv("wasb://sdcsparkrd01@asoscustprofdevelopment.blob.core.windows.net/" + fname, sep="\t")